In [4]:
import numpy as np
import tensorflow as tf
import argparse
import os
from six.moves import cPickle
import nltk
import time
import queue as Q
from operator import itemgetter
from model import Model
import itertools
from numpy.random import choice
from collections import defaultdict
from gensim.models.keyedvectors import KeyedVectors
import string
import re
import collections
import sys
import pickle
import getopt
from Traversal import *
from extra_func import *
from nltk.corpus import stopwords
import random

### RUN NEXT CELL

In [2]:
glove_model = KeyedVectors.load_word2vec_format('./storyline_for_reference/glove.6B.300d.word2vec.txt',binary=False)
  # system arguments
#topics = [sys.argv[1]]
try:
    seed = int(sys.argv[2])
except:
    seed = 1
np.random.seed(seed) # seed for reproducibility

num_per_topic = 1 
#topic=topics[0]
text_list = [("data\\frost\\input.txt","frost_model_forward"),("data\whitman\input.txt","whitman_model"),("data\ya\input.txt","ya_model")]
np.random.shuffle(text_list)
t = text_list[2][0] #THIS TEXT IS THE VOCAB!
save_dir = text_list[2][1] #THIS IS THE MODEL DIRECTORY
text = open(t)
text = text.read()
with open("./cmudict-0.7b.txt") as f:
    lines = [line.rstrip("\n").split() for line in f if (";;;" not in line)]
dictMeters = {}
for i in range(len(lines)):
    line = lines[i]
    newLine = [line[0].lower()]
    if("(" in newLine[0] and ")" in newLine[0]):
        newLine[0] = newLine[0][:-3]
    chars = ""
    for word in line[1:]:
        for ch in word:
            if(ch in "012"):
                if(ch == "2"):
                    chars+="1"
                else:
                    chars+=ch
    newLine+=[chars]
    lines[i] = newLine
    if(newLine[0] not in dictMeters): #THIS IF STATEMENT ALLOWS FOR MULTIPLE PRONUNCIATIONS OF A WORD
        dictMeters[newLine[0]]=[chars]
    else:
        if(chars not in dictMeters[newLine[0]]):
            dictMeters[newLine[0]]+=[chars]
words = [simple_clean(word) for word in text.split()]
uniques = set()
for word in words:
    if word not in dictMeters:
        continue
    else:
        uniques.add(word)
corpus = list(uniques)
dictCorpusMeterGroups = createMeterGroups(corpus,dictMeters)
dictWordTransitions = {}
#fsaLine = [State((0,i)) for i in range(11)]
dictMeters["i"]+=["0"]
dictMeters["the"]+=["0"]
glove_words = glove_model.vocab.keys() #SHOULDN'T THIS HAVE BEEN MOST COMMON WORDS IN THE CORPUS?
# CORPUS LIMITED TO WORDS IN GLOVE VOCAB
word_counts = collections.Counter([x for x in words if x in dictMeters])
corpi = [x[0] for x in word_counts.most_common() if x[0] in glove_words]
dictCorpusMeterGroups = createMeterGroups(corpi,dictMeters)
dictPartSpeechTags = createPartSpeechTags(corpi,dictMeters)
dictPossiblePartsSpeech = possiblePartsSpeechPaths()
#fsa_ret = formLinkedTree(0,dictMeters,corpi,fsaLine,dictWordTransitions,dictCorpusMeterGroups)
#fsa_row = fsa_ret[0]
  ########################
from collections import defaultdict
POS_terms= defaultdict(list)
for word, tag in dictPartSpeechTags.items():
    POS_terms[tag].append(word)
def get_possible_words(word, POS_terms):
    pos=nltk.pos_tag([word])[0][1]
    list_POS=dictPossiblePartsSpeech[pos]
    res=[]
    for x in list_POS:
        res.extend(POS_terms[x])
    return res
###################################### 
#post_stress=5
#pred_stress=5
#width = 50
#wordPools = [set([]) for n in range(4)]
#poem_1= genPoem('grapes', 'purple', save_dir,topic,width,wordPools)





### Specify the model to be used. DEFAULT=WHITMAN

In [64]:
from model import Model

poet=0 # CHANGE HERE FROST=0, WHITMAN=1



text_list = [("data\\frost\\input.txt","frost_model_forward"),("data\whitman\input.txt","whitman_model"),("data\ya\input.txt","ya_model")]
#np.random.shuffle(text_list)
t = text_list[poet][0] #THIS TEXT IS THE VOCAB!
save_dir = text_list[poet][1] #THIS IS THE MODEL DIRECTORY
tf.reset_default_graph()
with open(os.path.join(save_dir, 'config.pkl'), 'rb') as f:
    saved_args = cPickle.load(f)
with open(os.path.join(save_dir, 'words_vocab.pkl'), 'rb') as f:
    word_keys, vocab = cPickle.load(f)
model = Model(saved_args, True)

## Run this

In [151]:
def bool_init(word):
    postag=nltk.pos_tag([word])[0][1]
    if postag in set(['RB','RBR', 'RBS', 'VB', 'VBZ', 'LS','UH','PRP','MD']):
        return True
    return False
def init_word(vocabulary):
    init=random.choice(vocabulary)
    while bool_init(init):
        init=random.choice(vocabulary)
    return init

def bool_last_word(sentence):
    last=nltk.pos_tag(sentence)
    last=last[-1][1]
    if last not in set(['EX','JJ','JJR','JJS','WDT','WP','WRB','PRP','RP','TO']):
        return True
    return False

In [144]:
def generate_last(sess, model, vocab, prob_seq, sequence, state, temp, lists, nwords, width, rest_vocab):
    d,s=model.compute_fx(sess, vocab, prob_seq, sequence, state, temp)
    if len(sequence)<nwords:
        new_vocab=[vocab[x] for x in rest_vocab]
        d_new=d[new_vocab]
        max_ind=d_new.argsort()[-width:][::-1]
        for x in max_ind:
            new_prob=d[new_vocab[x]]
            new_word=list(vocab.keys())[new_vocab[x]]
            if(new_word == sequence[-1]):
                continue
            if(partsOfSpeechFilter(sequence[-1],new_word,dictPartSpeechTags,dictPossiblePartsSpeech)):
                continue
            generate_last(sess, model, vocab, new_prob, sequence+[new_word],s, temp, lists, nwords, width, rest_vocab)
    else:
        if bool_last_word(sequence):
            result=(prob_seq, sequence)
            return lists.append(result)
        else:
            return

In [66]:
f=print_five_words('chicken')

vertebrates->bird->fowl~~>person->human


In [139]:
def build_rest_vocab(definition1, definition2, vocabulary):
    result=simple_clean(definition1).split()
    result2=simple_clean(definition2).split()
    final=set(result+result2)
    final=list(final)+stopwords.words('english')
    return [x for x in final if x in set(list(vocabulary.keys()))]

## Function to create poem by using the synsets

#### Inputs: model, word to be used to construct the meta-poem, number of words in line

--It creates the 5 words required for the poem

-- check if all the words are in the vocab

--- Construct the lines by using pairs of words

In [124]:
def create_poem_2(model, seed, length_line):
    result=[]
    five=print_five_words(seed)
    five_def=create_pairs([x[1] for x in five])
    five_words=create_pairs([x[0] for x in five])
    print(' ')
    if bool_five_words([x[0] for x in five], vocab)==False:
        print ('WORDS NOT IN VOCAB, PLEASE INSERT A DIFFERENT SEED')
        return None
    else:
        with tf.Session() as sess:
            tf.global_variables_initializer().run()
            saver = tf.train.Saver(tf.global_variables())
            ckpt = tf.train.get_checkpoint_state(save_dir)
            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess, ckpt.model_checkpoint_path)
                saver.restore(sess, ckpt.model_checkpoint_path)
                state = sess.run(model.initial_state)
                init_score = np.array([[0]])
                list_pairs=create_pairs(five)
                for pair_word, pair_def in zip(five_words, five_def):
                    
                    tt_list=[]
                    res_vocab=build_rest_vocab(pair_def[0], pair_def[1], vocab)
                    res_vocab=res_vocab+pair_word
                    init=init_word(res_vocab)
                    generate_last(sess, model, vocab, init_score, [init], state, 1, tt_list,length_line, 20, res_vocab)
                    final_list=sorted(tt_list, key=lambda tup: tup[0], reverse=True)
                    result.append(' '.join(sampleLine(final_list, 10)))
        print('------POEM------')
        for x in result:
            print(x)
        print('-----------------')
        return result

In [158]:
def poem(length):
    i=0
    k=list(vocab.keys())
    random.shuffle(k)
    k=k[:100]
    word=k[i]
    try: 
        t=create_poem_2(model, word, lenth)
    except UnboundLocalError or ValueError or TypeError:
        t=None
    while t==None:
        i+=1
        word=k[i]
        try: 
            t=create_poem_2(model, word, length)
        except UnboundLocalError or ValueError or TypeError:
            t=None
        

In [ ]:
poem(4)

group->people->generations~~>study->inspection
 
WORDS NOT IN VOCAB, PLEASE INSERT A DIFFERENT SEED
tendency->elasticity->give~~>test->something
 
WORDS NOT IN VOCAB, PLEASE INSERT A DIFFERENT SEED
knowledge->awareness->sensation~~>motion->use
 
WORDS NOT IN VOCAB, PLEASE INSERT A DIFFERENT SEED
act->prayer->collect~~>develop->make
 
WORDS NOT IN VOCAB, PLEASE INSERT A DIFFERENT SEED
scope->expanse->space~~>space->expanse
 
WORDS NOT IN VOCAB, PLEASE INSERT A DIFFERENT SEED
sugar->brittle->be~~>be->brittle
 
INFO:tensorflow:Restoring parameters from frost_model_forward\model.ckpt-80000
INFO:tensorflow:Restoring parameters from frost_model_forward\model.ckpt-80000
was
i
when
